In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/25 16:23:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Load sparksql magic to execute SQL with spark

In [2]:
%load_ext sparksql_magic

In [3]:
%alias_magic --cell sql sparksql

Created `%%sql` as an alias for `%%sparksql`.


In [4]:
%%sql
CREATE TABLE student (
    id INT, 
    name STRING,
    `nestedwithspaces` STRUCT<`sub field`:STRUCT<`sub field2`:STRING>>,
    age INT,
    books ARRAY<STRUCT<`title`:STRING, `chapters`:ARRAY<STRUCT<`paragraph`:STRING>>>>,
    struct_col STRUCT<`address`:STRUCT<`streetName`:STRING, `streetNumber`:BIGINT>>,
    map_col MAP<STRING, MAP<STRING, STRUCT<`start`:BIGINT,`end`:BIGINT>>>

    ) USING PARQUET

### Compose and execute query using pyspark SQL

In [5]:
%%sql

SELECT
s.books.chapters.paragraph,

s.books['sfsdf'].chapters.paragraph as x,
s.nestedwithspaces.`.`sub field`.`sub field2`,
s.struct_col.address.streetName
s.books['The_Odyssey'].chapters[3].paragraph as a,
    s.nestedwithspaces.`sub field`.`sub field2`
FROM
    student AS s
WHERE
    array_contains(map_keys(s.books.chapters.paragraph), 'The_Odyssey') = TRUE

AnalysisException: cannot resolve 'map_keys(s.`books`)' due to data type mismatch: argument 1 requires map type, however, 's.`books`' is of array<struct<title:string,chapters:array<struct<paragraph:string>>>> type.; line 8 pos 19;
'Project ['s.books[The_Odyssey][chapters][3][paragraph] AS a#0, 's.nestedwithspaces.sub field.sub field2]
+- 'Filter ('array_contains(map_keys(books#5), The_Odyssey) = true)
   +- SubqueryAlias s
      +- SubqueryAlias spark_catalog.default.student
         +- Relation[id#1,name#2,nestedwithspaces#3,age#4,books#5,struct_col#6,map_col#7] parquet


In [14]:
%%sql

SELECT
    s.books.title,
    s.boo
FROM
    student AS s
WHERE
    array_contains(s.books.title, 'x')

title


### Compose and syntas highlight SQL in python string

In [21]:
sql_statement = '''
--start-sql-syntax
SELECT
    s.books.chapters[3].paragraph as a,
    s.id
FROM
    student AS s
WHERE
    array_contains(s.books.title, 'The_Odyssey')
--end-sql-syntax
'''

print(sql_statement)


--start-sql-syntax
SELECT
    s.books.chapters[3].paragraph as a,
    s.id
FROM
    student AS s
WHERE
    array_contains(s.books.title, 'The_Odyssey')
--end-sql-syntax



In [22]:
spark.sql(sql_statement).show()

+---+---+
|  a| id|
+---+---+
+---+---+

